# Examples on using the Basel Precision Instruments LNHR DAC II Telnet module

Copyright (c) Basel Precision Instruments GmbH (2024)

................................................................................................................

The LNHR DAC II provides a wide set of commands to get the most out of the device.

## 1 - imports and setup
For this example the Basel Precision Instruments LNHR DAC II Telnet driver is used (available on Github).

In [ ]:
import telnet

# only used for a custom made waveform
from math import sin
from math import cos

# create an instance of the LNHR DAC device
DAC = telnet.LNHRDAC("192.168.0.5",23)

## 2 - set a DC voltage on a channel
The simplest form of controlling the LNHR DAC II is setting the outputs manually in a python script to a DC voltage. For best noise performance, the low bandwith should be chosen.

In [ ]:
DAC.send_command("all off")

# set DAC channel 1 to 1 volt
DAC.send_command("1 8CCCCC") 
DAC.send_command("1 on")

# set multiple channels (2-6) to -2.5 volts
for channel in range(2, 5):
    DAC.send_command(f"{channel} 600000")
    DAC.send_command(f"{channel} lbw")
    DAC.send_command(f"{channel} on")

## 3 - update all channels simultaneously

The DAC has a synchronous update function, whith which all channels can be set to independent DC voltages at the same time. This update can be triggered by software or hardware.

In [ ]:
# change update mode to synchronous
DAC.send_command("c um-l 1")

# assign voltages to outputs
DAC.send_command("1 80F000;2 6FFFFF;3 80F000;4 6FFFFF;5 80F000;6 6FFFFF;7 80F000;8 6FFFFF;9 80F000;10 6FFFFF;11 80F000;12 6FFFFF")

# turn channels on
DAC.send_command("all on")

# apply voltages to outputs
DAC.send_command("c sync-l")

## 3 - create a standard waveform with the integrated standard waveform generator
Manually setting DC voltages will neither yield nice looking waveforms nor can the results be replicated easily. 

For standard waveforms the LNHR DAC II therefore provides a set of commands to easily create well defined waveforms:
- Sine, phaseshift and DC offset applicable
- Triangular, phaseshift and DC offset applicable
- Sawtooth, phaseshift and DC offset applicable
- Ramp, phaseshift and DC offset applicable
- Rectangular, duty-cycle, phaseshift and DC offset applicable
- Gaussian Noise (fixed and random)

In [ ]:
# create a simple sinewave on channel 13
DAC.send_command("13 off")
DAC.send_command("c awg-c stop")

while DAC.expect_query_answer("c awg-c ava?", "0"): # wait for availability of awg-c 
    pass

DAC.send_command("c awg-c ch 13") # select channel for awg
DAC.send_command("c swg mode 0") # generate new waveform
DAC.send_command("c swg wf 0") # choose waveform (0 = sine)
# further properties of waveform could be specified here, i.e. amplitude, offset or phase
DAC.send_command("c swg wmem 2") # select wave-memory to save the generated waveform into
DAC.send_command("c awg-c cs 0") # set number of cycles (0 = infinite cycles)
DAC.send_command("c swg wfun 0") # generated waveform will be copied to the selected wave-memory
DAC.send_command("c swg apply") # apply all changes to wave-memory now
DAC.send_command("c wav-c write") # write content of wave-memory into awg memory

print("writing to memory ...", end="")
while DAC.expect_query_answer("c wav-c busy?", "1"): # wait until waveform is written into awg memory
    print(".", end="")

DAC.send_command("c awg-c start") # start awg
DAC.send_command("13 on") #turn channel on

## 4 - create a custom waveform using mathematical functions
A completely arbitrary waveform can also be defined and stored into the LNHR DAC II memory. Limiting factors are the maximum memory size of 34000 points, the maximum update rate of 100 kHz (10 us) an d the set bandwidth.

In [ ]:
# create a custom waveform on channel 17
DAC.send_command("17 off")
DAC.send_command("c awg-d stop")

while DAC.expect_query_answer("c awg-d ava?", "0"): # wait for availability of awg-c 
    pass

DAC.send_command("c awg-d ch 17") # select channel for awg
DAC.send_command("c awg-d cs 0") # set number of cycles (0 = infinite cycles)
DAC.send_command("c wav-d clr") # clear wavememory

print("generating waveform ...", end="")
for x in range(0, 34000):
     y = 3*sin(x/200) + 2*cos(0.03*x) + (8/7)*sin(x/100) + 3.7*cos(x/10000) # generating arbitrary curve
     DAC.send_command(f"wav-a {x:x} {y:.6f}", hold_connection=True) # transmitting datapoints as voltage to DAC
     if x % 100 == 0: print(".", end="")
print("\n")

DAC.send_command("c wav-d write") # write content of wave-memory into awg memory

print("writing to memory ...", end="")
while DAC.expect_query_answer("c wav-c busy?", "1"): # wait until waveform is written into awg memory
    print(".", end="")

DAC.send_command("c awg-cd cp 1000") # set clock period, this might interfere with other awg's
DAC.send_command("c awg-d start") # start awg
DAC.send_command("17 on") #turn channel on